In [7]:
import os
import pickle
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import glob

In [8]:
class SIFt(object):
    def __init__(self, pickled_sift):
        
        if pickled_sift is None:
            print("ohno")
            return None

        """
        Initialize an SIFt class.
        Parameters:
            fn_pro_PDB - PDB file name of the protein
            fn_lig_PDB - PDB file name of the ligand
            fn_pro_MOL2 - MOL2 file name of the protein
            fn_lig_MOL2 - MOL2 file name of the ligand
            ID - ID of the complex
            addH - whether to add hydrogen atoms when reading in the structure file
            sant - whether to sanitize the molecule when reading in the structure file
            int_cutoff - distance threshold for identifying protein-ligand interacting atoms 
        """
        self.ID = pickled_sift.ID 
       # print('Constructing an SIFt object for %s.........\n' % self.ID)
        # read in pdb coordinates and topology
        self.lig = pickled_sift.lig
        self.pro = pickled_sift.pro

 
       
        # parse protein pdb file for identifying sidechain/mainchain atoms
#        parser = PDBParser()
#        self.structure = parser.get_structure(self.ID, fn_pro_PDB)
#        self.chid = self.pro[1].GetAtomWithIdx(0).GetPDBResidueInfo().GetChainId()
        # identify interacting area
        self.contact_bins = pickled_sift.contact_bins
        self.pd = pickled_sift.pd
        self.cont = pickled_sift.cont
        self.contacts = pickled_sift.contacts

        self.protein_env = pickled_sift.protein_env
        self.ligand_env = pickled_sift.ligand_env

In [9]:
def load_descriptors(csv_path, embeddings_root):
    """
    Load descriptors and labels from a CSV file and embeddings folder structure.

    CSV must have columns: 'protein_name', 'binding_affinity'.
    For each protein_name, expects files:
        embeddings_root/protein_name/protein_esm.pkl
        embeddings_root/protein_name/ligand_embedding.pkl
    """
    df = pd.read_csv(csv_path)
    X_list, y_list = [], []

    for _, row in df.iterrows():
        pname = row['id']
        affinity = row['affinity']
        subfolder = os.path.join(embeddings_root, pname)
        

        esm_file = os.path.join(subfolder, 'protein_esm.pkl')
        lig_file = os.path.join(subfolder, 'ligand_embedding.pkl')

        # Load embeddings
        try:
            with open(esm_file, 'rb') as f:
                prot_vec = pickle.load(f)
            with open(lig_file, 'rb') as f:
                lig_vec = pickle.load(f)
        except:
            continue
        # Concatenate descriptors
        descriptor = np.concatenate([prot_vec, lig_vec])

        X_list.append(descriptor)
        y_list.append(affinity)

    X = np.vstack(X_list)
    y = np.array(y_list, dtype=float)
    return X, y



In [10]:

def cross_validate_models(X, y, n_splits=10, random_state=42):
    """
    Perform K-Fold CV for three regression models and return CV metrics.
    """
    models = {
        # 'RandomForest1': RandomForestRegressor(random_state=random_state,n_estimators = 100),
        # 'RandomForest2': RandomForestRegressor(random_state=random_state,n_estimators = 200),
        # 'RandomForest3': RandomForestRegressor(random_state=random_state,n_estimators = 300),
        # 'RandomForest4': RandomForestRegressor(random_state=random_state,n_estimators = 400),
        # 'RandomForest5': RandomForestRegressor(random_state=random_state,n_estimators = 450),
        # 'RandomForest6': RandomForestRegressor(random_state=random_state,n_estimators = 500),
        # 'RandomForest7': RandomForestRegressor(random_state=random_state,n_estimators = 550),
        'RandomForest8': RandomForestRegressor(random_state=random_state,n_estimators = 600),
        # 'RandomForest9': RandomForestRegressor(random_state=random_state,n_estimators = 650),
        # 'RandomForest10': RandomForestRegressor(random_state=random_state,n_estimators = 700),   
        # 'GradientBoosting1': GradientBoostingRegressor(random_state=random_state,n_estimators=100),
        # 'GradientBoosting2': GradientBoostingRegressor(random_state=random_state,n_estimators=200),
        # 'GradientBoosting3': GradientBoostingRegressor(random_state=random_state,n_estimators=300),
        # 'GradientBoosting4': GradientBoostingRegressor(random_state=random_state,n_estimators=400),
        # 'GradientBoosting5': GradientBoostingRegressor(random_state=random_state,n_estimators=450),
        # 'GradientBoosting6': GradientBoostingRegressor(random_state=random_state,n_estimators=500),
        # 'GradientBoosting7': GradientBoostingRegressor(random_state=random_state,n_estimators=550),
        # 'GradientBoosting8': GradientBoostingRegressor(random_state=random_state,n_estimators=600),
        # 'GradientBoosting9': GradientBoostingRegressor(random_state=random_state,n_estimators=700),
        # "svr_poly1" : SVR(kernel='poly', degree=4, C=1.0, epsilon=0.1, coef0=1),
        # "svr_poly2" : SVR(kernel='poly', degree=4, C=1.0, epsilon=0.1, coef0=2),
        # "svr_poly3" : SVR(kernel='poly', degree=4, C=1.0, epsilon=0.1, coef0=3),
    }
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    cv_results = {name: {'pearson': [], 'rmse': []} for name in models}

    for name, model in models.items():
        print("doing",name)
        for train_idx, val_idx in kf.split(X):
            print(len(val_idx))
            X_tr, X_val = X[train_idx], X[val_idx]
            y_tr, y_val = y[train_idx], y[val_idx]
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_val)
            # Metrics
            r, _ = pearsonr(y_val, y_pred)
            rmse = np.sqrt(mean_squared_error(y_val, y_pred))
            cv_results[name]['pearson'].append(r)
            cv_results[name]['rmse'].append(rmse)

        # Aggregate
        pearson_vals = cv_results[name]['pearson']
        rmse_vals    = cv_results[name]['rmse']

        pearson_mean = np.mean(pearson_vals)
        pearson_std  = np.std(pearson_vals, ddof=1)   # sample std
        rmse_mean    = np.mean(rmse_vals)
        rmse_std     = np.std(rmse_vals, ddof=1)      # sample std

        print(
            f"{name} CV -> "
            f"Pearson: {pearson_mean:.3f} ± {pearson_std:.3f}, "
            f"RMSE: {rmse_mean:.3f} ± {rmse_std:.3f}"
        )


    return cv_results, models



In [11]:

def train_and_test(train_csv, train_root, test_csv, test_root):
    # Load training data
    X_train, y_train = load_descriptors(train_csv, train_root)
    print(f"Training samples: {X_train.shape[0]}, features: {X_train.shape[1]}")

    # Cross-validate
    cv_results, models = cross_validate_models(X_train, y_train)

    # Train final models on all training data and evaluate on test set
    X_test, y_test = load_descriptors(test_csv, test_root)
    print(f"Test samples: {X_test.shape[0]}")

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        r, _ = pearsonr(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        print(f"{name} Test -> Pearson: {r:.3f}, RMSE: {rmse:.3f}")




In [ ]:
if __name__ == '__main__':
    # Paths to CSVs and embedding folders
    train_csv_path = '/home/ansh-meshram/Desktop/work/bio_project/PDBbind/indexes/rs_index.csv'       # CSV with 'protein_name' and 'binding_affinity'
    train_emb_folder = '/home/ansh-meshram/Desktop/work/bio_project/PDBbind/data'   # Folder containing subfolders per protein
    test_csv_path = '/home/ansh-meshram/Desktop/work/bio_project/PDBbind/indexes/casf2016_index.csv'
    test_emb_folder = '/home/ansh-meshram/Desktop/work/bio_project/PDBbind/data'

    train_and_test(train_csv_path, train_emb_folder, test_csv_path, test_emb_folder)


Training samples: 4826, features: 384
doing RandomForest8
483
